### Maximal Marginal Relevance (MMR)
"How can we pick results that are not only relevant to the query but also different from each other?"
* MMR is an information retrieval algorithm designed to reduce redundancy in the retrieved results while maintaining high relevance to the query.

#### Why MMR Retriever?

In regular similarity search, you may get documents that are:
* All very similar to each other
* Repeating the same info
* Lacking diverse perspectives

MMR Retriever avoids that by:
* Picking the most relevant document first
* Then picking the nect most relevant and least similar to already selected docs
* And so on...

This helps especially in RAG pipelines where:
* You want your context window to contain diverse but still relevant information
* Especially useful when documents are semantically overlapping

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [6]:
# Initialize embedding model
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [7]:
# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [8]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [9]:
query = "What is langchain?"
results = retriever.invoke(query)

In [10]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
Embeddings are vector representations of text.

--- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.
